In [0]:
from pyspark.sql.types import *

orders_schema = StructType([
    StructField("order_id", LongType(), True),
    StructField("customer_id", LongType(), True),
    StructField("customer_fname", StringType(), True),
    StructField("customer_lname", StringType(), True),
    StructField("customer_city", StringType(), True),
    StructField("customer_state", StringType(), True),
    StructField("customer_pincode", LongType(), True),
    StructField("order_items", ArrayType(StructType([
        StructField("order_item_id", LongType(), True),
        StructField("order_item_product_id", LongType(), True),
        StructField("order_item_product_price", DecimalType(10, 2), True),
        StructField("order_item_quantity", LongType(), True),
        StructField("order_item_subtotal", DecimalType(10, 2), True)
    ])), True)
])


Creating a directory

In [0]:
dbutils.fs.mkdirs("dbfs:/FileStore/processed/updatemode/")

Out[12]: True

In [0]:
orders_df = spark \
    .readStream \
        .format("json") \
            .schema(orders_schema) \
                .option("path","dbfs:/FileStore/merge.json") \
                    .load()

In [0]:
from pyspark.sql.types import *

schema = StructType([
    StructField("customer_id", LongType(), True),
    StructField("orders_placed", LongType(), False),
    StructField("product_purchased", LongType(), False),
    StructField("amount_spent", DecimalType(20, 2), True)
])


In [0]:
aggregated_orders = spark \
    .readStream \
        .format("csv") \
            .schema(schema) \
                .option("path","dbfs:/FileStore/processed/aggregated_orders.csv") \
                    .load()





---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2043068789303967>:1
----> 1 aggregated_orders = spark \
      2     .readStream \
      3         .format("csv") \
      4             .schema(schema) \
      5                 .option("path","dbfs:/FileStore/processed/aggregated_orders.csv") \
      6                     .load().show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:920, in DataFrame.show(self, n, truncate, vertical)
    914     raise PySparkTypeError(
    915         error_class="

In [0]:
spark.sql("create table orders_final_result1 (customerid long,orders_placed long ,products_purchased long, amount_apent float)")

Out[22]: DataFrame[]

In [0]:
def Myfunction(orders_result,batch_id):
    order_result.createOrReplaceTempView("orders_result")
    merge_statement ="""merge into orders_final_result1 using orders_result s 
    on t.customer_id == s.customer_id
    when matched then
    update set t.productrs_purchased = s.products_purchased,
               t.orders_placed =s.orders_placed,
               t.amount_spent= s.amount_spent
               when not matched then
               insert * """

    
    orders_result._jdf.sparkSession().sql(merge_statement)

In [0]:
aggregated_orders.show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2043068789303974>:1
----> 1 aggregated_orders.show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:920, in DataFrame.show(self, n, truncate, vertical)
    914     raise PySparkTypeError(
    915         error_class="NOT_A_BOOLEAN",
    916         message_parameters={"arg_name": "vertical", "arg_type": type(vertical).__name__},
    917     )
    919 if isinstance(truncate, bool) and truncate:
--> 920     print(self._jdf.showString(n, 20, vertical)

In [0]:
streaming_query = aggregated_orders \
    .writeStream \
        .format("delta") \
            .outputMode("update") \
                .option("checkPointLocation","checkpointdir104") \
                    .foreachBatch(Myfunction)


In [0]:
spark.sql(""" select * from orders_final_result1""").show()

+----------+-------------+------------------+------------+
|customerid|orders_placed|products_purchased|amount_apent|
+----------+-------------+------------------+------------+
+----------+-------------+------------------+------------+

